In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

## Data

### Read Data

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



### Tokenization

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [7]:
print(encode("Helllo! What's up?"))
print(decode([20, 43, 50, 50, 50, 53, 2, 1, 35, 46, 39, 58, 5, 57, 1, 59, 54, 12]
))

[20, 43, 50, 50, 50, 53, 2, 1, 35, 46, 39, 58, 5, 57, 1, 59, 54, 12]
Helllo! What's up?


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)

In [9]:
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


### Train/Val Split

In [10]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

### Data Loader

In [11]:
block_size = 8
train_data[: block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Input:{context}, Target:{target}")

Input:tensor([18]), Target:47
Input:tensor([18, 47]), Target:56
Input:tensor([18, 47, 56]), Target:57
Input:tensor([18, 47, 56, 57]), Target:58
Input:tensor([18, 47, 56, 57, 58]), Target:1
Input:tensor([18, 47, 56, 57, 58,  1]), Target:15
Input:tensor([18, 47, 56, 57, 58,  1, 15]), Target:47
Input:tensor([18, 47, 56, 57, 58,  1, 15, 47]), Target:58


In [13]:
torch.manual_seed(1337)
batch_size = 4 # how many independdent sequences will we process in parallel
block_size = 8 # what is max context length for predictions

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size):
  for t in range(block_size):
      context = xb[b, :t+1]
      target = yb[b, t]
      print(f"Input:{context.tolist()}, Target:{target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
Input:[24], Target:43
Input:[24, 43], Target:58
Input:[24, 43, 58], Target:5
Input:[24, 43, 58, 5], Target:57
Input:[24, 43, 58, 5, 57], Target:1
Input:[24, 43, 58, 5, 57, 1], Target:46
Input:[24, 43, 58, 5, 57, 1, 46], Target:43
Input:[24, 43, 58, 5, 57, 1, 46, 43], Target:39
Input:[44], Target:53
Input:[44, 53], Target:56
Input:[44, 53, 56], Target:1
Input:[44, 53, 56, 1], Target:58
Input:[44, 53, 56, 1, 58], Target:46
Input:[44, 53, 56, 1, 58, 46], Target:39
Input:[44, 53, 56, 1, 58, 46, 39], Target:58
Input:[44, 53, 56, 1, 58, 46, 39, 58], Target:1
Input:[52], Target:58
Input:[52, 58], Target

## Models

### 1. Simple Baseline

In [22]:
class BigramLM(nn.Module):
    
    def __init__(self, vocab_size):
      super().__init__()
      self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
      logits = self.token_embedding_table(idx) # (B, T, C)
      if targets is None:
        loss = None
      else:
        B,T,C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
      return logits, loss
    
    def generate(self, idx, max_new_tokens):
      # idx is (B,T) array of indices in the current context
      for _ in range(max_new_tokens):
        logits, loss = self(idx) # get predictions
        logits = logits[:, -1, :] # focus only on last time step, becomes (B,C)
        probs = F.softmax(logits, dim=1) # (B,C)
        idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
        idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
      return idx
        

m = BigramLM(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([256, 65])
tensor(4.6305, grad_fn=<NllLossBackward0>)


In [23]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(
    m.generate(idx, max_new_tokens=100)[0].tolist()
))


WdJ??knj,iuX;XoMdl.'
i,MrP'FBkJczOtLrGDs,xZJsnqKrEePe!GIxsycv3xFAmwnyfop?tYduIXkq&BPbo!lprv QN:lIu?'


In [24]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [25]:
batch_size = 32
for step in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if step % 1000 == 0: print(loss.item())


4.624002456665039
3.7287235260009766
3.1125755310058594
2.796713352203369
2.69694185256958
2.457777738571167
2.5532517433166504
2.4706366062164307
2.526153326034546
2.482177257537842


In [32]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(
    m.generate(idx, max_new_tokens=500)[0].tolist()
))




ERomyontie.yon loy ghed.
AMain tsowarcThandyoucabe, we sudourd'seameerilil d inoind ais w V!

KICLethe deariseee w d pand al m nouesagh, ngigeen, Wh thetilurd c-helwowinom mathar, mas my wet hud; mbe t: meethake; g welinor, w hast tho and.
Tovealeier der ave s t en
DUS:
Walty hens,
QUS yolo inoti m to pre mu moune s
Anoercees s dousome oun nd;3ThEERConleacesowhnchinf that t oard sh

ADUpPr thtal erant mye's t t fr we mime at posheser--uteout a move ines whilenters, Voo dsen;
weinchay, bu rale 


### 2. Mathematical Trick in Self-Attention

#### v1

In [14]:
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [15]:
# We want x[b,t] = mean_{i <= t} x[b, i]
xbow = torch.zeros((B,T,C)) # bow: bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0)

In [16]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [17]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

#### v2: Using the trick (batch matrix multiply) explained below:

In [18]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T,T) @ (B,T,C) --> (B,T,T) @ (B,T,C) ---> (B,T,C)

print('wei=')
print(wei)
print('---')
print('xbow2=')
print(xbow2)
print('---')
torch.allclose(xbow, xbow2)

wei=
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
---
xbow2=
tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0

True

The Trick

In [19]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [20]:
a = torch.tril(torch.ones(3,3))
a = a / a.sum(1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()

c = a @ b

print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[8., 6.],
        [5., 2.],
        [4., 4.]])
---
c=
tensor([[8.0000, 6.0000],
        [6.5000, 4.0000],
        [5.6667, 4.0000]])


#### v3:Using Softmax

In [21]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) # to indicate that tokens from future can not contribute
print(wei)
wei = F.softmax(wei, dim=1)
print(wei)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [23]:
xbow3 = wei @ x
print(xbow3)
torch.allclose(xbow, xbow3)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


True

#### v4: Self-Attention

In [2]:
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

In [3]:
# let's see single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

out = wei @ x

out.shape

torch.Size([4, 8, 32])